In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style('whitegrid')

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style('whitegrid')

In [ ]:
df = pd.read_csv("../../origin_data_minfrag/dataset.csv")
df

In [ ]:
def calculate_molecular_properties(smiles):
    """
    SMILES文字列から分子の物理化学的性質を計算する
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None, None, None, None, None
        
        # 水素結合ドナー数
        hbd = rdMolDescriptors.CalcNumHBD(mol)
        
        # 水素結合アクセプター数
        hba = rdMolDescriptors.CalcNumHBA(mol)
        
        # LogP (脂溶性)
        logp = Descriptors.MolLogP(mol)
        
        # 分子量
        mw = Descriptors.MolWt(mol)
        
        # 環の数
        ring_count = rdMolDescriptors.CalcNumRings(mol)
        
        return hbd, hba, logp, mw, ring_count
    except:
        return None, None, None, None, None

In [ ]:
# REF-SMILESの分子性質を計算
print("REF-SMILESの分子性質を計算中...")
ref_properties = df['REF-SMILES'].apply(calculate_molecular_properties)

# 結果を個別のカラムに分割
df['REF_HBD'] = ref_properties.apply(lambda x: x[0] if x[0] is not None else np.nan)
df['REF_HBA'] = ref_properties.apply(lambda x: x[1] if x[1] is not None else np.nan)
df['REF_LogP'] = ref_properties.apply(lambda x: x[2] if x[2] is not None else np.nan)
df['REF_MW'] = ref_properties.apply(lambda x: x[3] if x[3] is not None else np.nan)
df['REF_Rings'] = ref_properties.apply(lambda x: x[4] if x[4] is not None else np.nan)

# PRB-SMILESの分子性質を計算
print("PRB-SMILESの分子性質を計算中...")
prb_properties = df['PRB-SMILES'].apply(calculate_molecular_properties)

# 結果を個別のカラムに分割
df['PRB_HBD'] = prb_properties.apply(lambda x: x[0] if x[0] is not None else np.nan)
df['PRB_HBA'] = prb_properties.apply(lambda x: x[1] if x[1] is not None else np.nan)
df['PRB_LogP'] = prb_properties.apply(lambda x: x[2] if x[2] is not None else np.nan)
df['PRB_MW'] = prb_properties.apply(lambda x: x[3] if x[3] is not None else np.nan)
df['PRB_Rings'] = prb_properties.apply(lambda x: x[4] if x[4] is not None else np.nan)

print("計算完了！")

In [ ]:
# 新しく追加されたカラムを確認
new_columns = ['REF_HBD', 'REF_HBA', 'REF_LogP', 'REF_MW', 'REF_Rings',
               'PRB_HBD', 'PRB_HBA', 'PRB_LogP', 'PRB_MW', 'PRB_Rings']

print("追加されたカラム:")
for col in new_columns:
    print(f"{col}: {df[col].describe()}")
    print()

In [ ]:
# オプション: PRBとREFの差分も計算（構造活性相関解析に有用）
df['DIFF_HBD'] = df['PRB_HBD'] - df['REF_HBD']
df['DIFF_HBA'] = df['PRB_HBA'] - df['REF_HBA']
df['DIFF_LogP'] = df['PRB_LogP'] - df['REF_LogP']
df['DIFF_MW'] = df['PRB_MW'] - df['REF_MW']
df['DIFF_Rings'] = df['PRB_Rings'] - df['REF_Rings']

print("差分カラムも追加されました：")
diff_columns = ['DIFF_HBD', 'DIFF_HBA', 'DIFF_LogP', 'DIFF_MW', 'DIFF_Rings']
print(df[diff_columns].describe())

In [ ]:
# tanimoto類似度も計算してコラムに追加
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

def calculate_tanimoto(smiles1, smiles2):
    """
    2つのSMILES文字列からTanimoto類似度を計算する
    """
    try:
        mol1 = Chem.MolFromSmiles(smiles1)
        mol2 = Chem.MolFromSmiles(smiles2)
        if mol1 is None or mol2 is None:
            return np.nan
        
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, radius=2, nBits=2048)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, radius=2, nBits=2048)
        
        tanimoto = TanimotoSimilarity(fp1, fp2)
        return tanimoto
    except:
        return np.nan
    
print("Tanimoto類似度を計算中...")
df['Tanimoto'] = df.apply(lambda row: calculate_tanimoto(row['REF-SMILES'], row['PRB-SMILES']), axis=1)
print("計算完了！")
print(df['Tanimoto'].describe())

In [ ]:
# 最終的なデータフレームを表示
print(f"データフレームの形状: {df.shape}")
print(f"\n全カラム名:")
print(df.columns.tolist())
print(f"\n最初の5行:")
df.head()
df.to_csv("../../origin_data_minfrag/dataset_with_properties.csv", index=False)

# 特性値と活性との相関

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../../origin_data_minfrag/dataset_consistentsmiles_with_properties.csv')
df

In [ ]:
# DIFF系カラムの分布をプロット
diff_columns = ['DIFF_HBD', 'DIFF_HBA', 'DIFF_LogP', 'DIFF_MW', 'DIFF_Rings', 'Tanimoto']
diff_labels = ['Difference in HBD', 'Difference in HBA', 'Difference in LogP', 'Difference in MW', 'Difference in Rings', 'Tanimoto']

# 各分子性質に応じたbin数を設定
bin_counts = [13, 15, 41, 51, 11, 21]  # HBD, HBA, LogP, MW, Rings, Tanimoto

# 5つのサブプロットを作成
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Distribution of Pharmacophore Feature Differences', fontsize=24, fontweight='bold')


# プロット位置のリスト
plot_positions = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

for i, (col, label, pos, bins) in enumerate(zip(diff_columns, diff_labels, plot_positions, bin_counts)):
    row, col_idx = pos
    ax = axes[row, col_idx]
    
    # NaNを除外してプロット
    data = df[col].dropna()
    
    # ヒストグラム（各図で異なるbin数）
    ax.hist(data, bins=bins, alpha=0.7, color=f'C{i}', edgecolor='black')
    ax.set_title(f'{label}\n(Std Dev: {data.std():.2f})', fontsize=20)
    # ax.set_xlabel('Difference Value')
    ax.set_ylabel('Pairs', fontsize=18)
    ax.grid(True, alpha=0.3)
    
    # 軸の数値のフォントサイズを設定
    ax.tick_params(axis='x', labelsize=16)
    ax.tick_params(axis='y', labelsize=16)
    
    # 0の位置に縦線を追加
    ax.axvline(x=0, color='red', linestyle='--', alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

In [ ]:
# データセットの基本統計情報
print("=== Dataset Basic Statistics ===")
print(f"Total number of records: {len(df)}")
print(f"Number of unique AIDs: {df['AID'].nunique()}")
print(f"Number of unique TIDs: {df['TID'].nunique()}")
print()

# REF-SMILESとPRB-SMILESの統計
print("=== Molecule Statistics ===")
print(f"Number of unique REF-SMILES: {df['REF-SMILES'].nunique()}")
print(f"Number of unique PRB-SMILES: {df['PRB-SMILES'].nunique()}")
print()

# 全分子（REF + PRB）のunique数を計算
all_smiles = pd.concat([df['REF-SMILES'], df['PRB-SMILES']])
unique_molecules = all_smiles.nunique()
print(f"Total unique molecules (REF + PRB combined): {unique_molecules}")
print()

# AIDとTIDの組み合わせ統計
unique_aid_tid = df[['AID', 'TID']].drop_duplicates()
print(f"Number of unique AID-TID combinations: {len(unique_aid_tid)}")
print()

# 各AIDあたりの平均TID数
tid_per_aid = df.groupby('AID')['TID'].nunique()
print(f"Average number of TIDs per AID: {tid_per_aid.mean():.2f}")
print(f"Min TIDs per AID: {tid_per_aid.min()}")
print(f"Max TIDs per AID: {tid_per_aid.max()}")
print()

# 各TIDあたりの平均ペア数
pairs_per_tid = df.groupby(['AID', 'TID']).size()
print(f"Average number of pairs per TID: {pairs_per_tid.mean():.2f}")
print(f"Min pairs per TID: {pairs_per_tid.min()}")
print(f"Max pairs per TID: {pairs_per_tid.max()}")

In [ ]:
# pIC50活性差（delta_value）の分布をプロット
plt.figure(figsize=(10, 6))

# NaNを除外してプロット
delta_data = df['delta_value'].dropna()

# ヒストグラム
plt.hist(delta_data, bins=41, alpha=0.7, color='green', edgecolor='black')
plt.title(f'Distribution of ΔpIC50\n(Std Dev: {delta_data.std():.3f})', fontsize=20, fontweight='bold')
plt.xlabel('ΔpIC50', fontsize=16)
plt.ylabel('Pairs', fontsize=16)
plt.grid(True, alpha=0.3)

# 重要な閾値に縦線を追加
plt.axvline(x=0, color='red', linestyle='--', alpha=0.8, linewidth=2, label='No difference (0)')
plt.axvline(x=0.3, color='orange', linestyle='--', alpha=0.8, linewidth=2, label='Positive threshold (+0.3)')
plt.axvline(x=-0.3, color='orange', linestyle='--', alpha=0.8, linewidth=2, label='Negative threshold (-0.3)')

# 凡例を追加
plt.legend(fontsize=12)

plt.tight_layout()
plt.show()

# 統計情報を表示
print(f"=== pIC50 Activity Difference Statistics ===")
print(f"Total number of pairs: {len(delta_data)}")
print(f"Minimum delta_value: {delta_data.min():.3f}")
print(f"Maximum delta_value: {delta_data.max():.3f}")
print()

# 閾値による分類
positive_strong = (delta_data >= 0.3).sum()
positive_weak = ((delta_data > 0) & (delta_data < 0.3)).sum()
no_change = (delta_data == 0).sum()
negative_weak = ((delta_data < 0) & (delta_data > -0.3)).sum()
negative_strong = (delta_data <= -0.3).sum()

print(f"Activity improvement (delta_value >= 0.3): {positive_strong} pairs ({positive_strong/len(delta_data)*100:.1f}%)")
print(f"Weak improvement (0 < delta_value < 0.3): {positive_weak} pairs ({positive_weak/len(delta_data)*100:.1f}%)")
print(f"No change (delta_value = 0): {no_change} pairs ({no_change/len(delta_data)*100:.1f}%)")
print(f"Weak degradation (-0.3 < delta_value < 0): {negative_weak} pairs ({negative_weak/len(delta_data)*100:.1f}%)")
print(f"Activity degradation (delta_value <= -0.3): {negative_strong} pairs ({negative_strong/len(delta_data)*100:.1f}%)")

In [ ]:
# ファーマコフォア特徴とpIC50活性差の相関分析
from scipy.stats import pearsonr

# 分析に使用するカラム
diff_columns = ['DIFF_HBD', 'DIFF_HBA', 'DIFF_LogP', 'DIFF_MW', 'DIFF_Rings', 'Tanimoto']
diff_labels = ['ΔHydrogen Bond Donors', 'ΔHydrogen Bond Acceptors', 'ΔLogP', 'ΔMolecular Weight', 'ΔNumber of Rings', 'Tanimoto Similarity']

# ΔpIC50の絶対値を計算
df['abs_delta_value'] = df['delta_value'].abs()

# 散布図を作成
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Correlation between Pharmacophore Feature Changes and |ΔpIC50|', fontsize=24, fontweight='bold')


# プロット位置のリスト
plot_positions = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

correlation_results = {}

for i, (col, label, pos) in enumerate(zip(diff_columns, diff_labels, plot_positions)):
    row, col_idx = pos
    ax = axes[row, col_idx]
    
    # NaNを除外してデータを準備
    mask = df[col].notna() & df['abs_delta_value'].notna()
    x_data = df.loc[mask, col]
    y_data = df.loc[mask, 'abs_delta_value']
    
    # 散布図
    ax.scatter(x_data, y_data, alpha=0.01, s=30, color=f'C{i}')
    
    # 相関係数を計算
    if len(x_data) > 1:
        corr_coef, p_value = pearsonr(x_data, y_data)
        correlation_results[col] = {'correlation': corr_coef, 'p_value': p_value}
        
        # タイトルに相関係数を含める
        significance = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''
        ax.set_title(f'{label}\nr = {corr_coef:.3f}{significance}', fontsize=16)
    else:
        ax.set_title(f'{label}\nInsufficient data', fontsize=16)
    
    ax.set_xlabel(label, fontsize=14)
    ax.set_ylabel('|ΔpIC50|', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    # 0の位置に参照線を追加
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax.axvline(x=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# 相関結果をテーブル形式で表示
print("\n=== Correlation Analysis Results ===")
print(f"{'Feature':<25} {'Correlation':<12} {'P-value':<12} {'Significance':<12}")
print("-" * 65)

for i, (col, label) in enumerate(zip(diff_columns, diff_labels)):
    if col in correlation_results:
        corr = correlation_results[col]['correlation']
        p_val = correlation_results[col]['p_value']
        
        # 有意性を判定
        if p_val < 0.001:
            sig = '***'
        elif p_val < 0.01:
            sig = '**'
        elif p_val < 0.05:
            sig = '*'
        else:
            sig = 'n.s.'
        
        print(f"{label:<25} {corr:>8.3f}    {p_val:>8.3e}    {sig:<12}")

print("\n*** p < 0.001, ** p < 0.01, * p < 0.05, n.s. = not significant")

In [ ]:
# より詳細な相関分析：活性向上/低下別の解析
print("=== Detailed Correlation Analysis by Activity Change Direction ===")

# データを活性変化の方向で分割
mask_valid = df['delta_value'].notna()
improved_mask = mask_valid & (df['delta_value'] > 0)
degraded_mask = mask_valid & (df['delta_value'] < 0)

improved_data = df[improved_mask]
degraded_data = df[degraded_mask]

print(f"Activity improved pairs: {len(improved_data)}")
print(f"Activity degraded pairs: {len(degraded_data)}")
print()

# 各グループでの相関分析
print(f"{'Feature':<25} {'All Data':<12} {'Improved':<12} {'Degraded':<12}")
print("-" * 65)

for col, label in zip(diff_columns, diff_labels):
    # 全データ
    mask_all = df[col].notna() & df['delta_value'].notna()
    if mask_all.sum() > 1:
        corr_all, _ = pearsonr(df.loc[mask_all, col], df.loc[mask_all, 'delta_value'])
    else:
        corr_all = np.nan
    
    # 活性向上データ
    mask_imp = improved_data[col].notna()
    if mask_imp.sum() > 1:
        corr_imp, _ = pearsonr(improved_data.loc[mask_imp, col], improved_data.loc[mask_imp, 'delta_value'])
    else:
        corr_imp = np.nan
    
    # 活性低下データ
    mask_deg = degraded_data[col].notna()
    if mask_deg.sum() > 1:
        corr_deg, _ = pearsonr(degraded_data.loc[mask_deg, col], degraded_data.loc[mask_deg, 'delta_value'])
    else:
        corr_deg = np.nan
    
    print(f"{label:<25} {corr_all:>8.3f}    {corr_imp:>8.3f}    {corr_deg:>8.3f}")

In [ ]:
# ヒートマップで相関行列を可視化
# 分析用のデータフレームを作成
analysis_df = df[['delta_value'] + diff_columns].dropna()

# 相関行列を計算
correlation_matrix = analysis_df.corr()

# ヒートマップを作成
plt.figure(figsize=(10, 8))

# カラムラベルを日本語に変換
label_mapping = {
    'delta_value': 'ΔpIC50',
    'DIFF_HBD': 'ΔHydrogen Bond Donors',
    'DIFF_HBA': 'ΔHydrogen Bond Acceptors',
    'DIFF_LogP': 'ΔLogP',
    'DIFF_MW': 'ΔMolecular Weight',
    'DIFF_Rings': 'ΔNumber of Rings'
}

# 相関行列のインデックスとカラムを変更
correlation_matrix_labeled = correlation_matrix.rename(index=label_mapping, columns=label_mapping)

# ヒートマップ
mask = np.triu(np.ones_like(correlation_matrix_labeled, dtype=bool), k=1)
sns.heatmap(correlation_matrix_labeled, 
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            fmt='.3f',
            square=True,
            mask=mask,
            cbar_kws={'label': 'Pearson Correlation Coefficient'},
            annot_kws={'size': 12})

plt.title('Correlation Matrix: Pharmacophore Features vs ΔpIC50', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print(f"\nCorrelation analysis completed using {len(analysis_df)} valid data points.")